In [13]:
import pandas as pd
from datetime import datetime, timedelta

from classes import Visitor, Entrance, Attraction, Seasonal, Utility, ThemePark

In [14]:
#STEP 1: SPAWNING VISITORS, MAKE visitor_generator_df

spawning_df = pd.read_csv('../data/spawning.csv')

# Convert columns to the desired types
spawning_df['Average Wait Time'] = spawning_df['Average Wait Time'].astype(float)  # Convert to float
spawning_df['Number of Visitors'] = spawning_df['Number of Visitors'].astype(int)  # Convert to int
spawning_df['Number of New Visitors Arriving'] = spawning_df['Number of New Visitors Arriving'].astype(int)  # Convert to int

# Convert df into dictionary for easier reference
spawning_dict = {}
for index, row in spawning_df.iterrows():
    number = row['Number of New Visitors Arriving']
    if number >= 0:
        spawning_dict[row['Time']] = number
    else:
        spawning_dict[row['Time']] = 0

In [15]:
#STEP 2: IMPORT LIST OF ITINERARIES
itineraries = pd.read_csv('../data/itinerary.csv')
itineraries['path list'] = itineraries['path'].str.split()
all_possible_itineraries = [sublist for sublist in itineraries['path list']]

In [16]:
#STEP 3: CREATING NODES, MAKE NODE LIST 

attraction_generator_df = pd.read_csv("../data/theme_park_nodes.csv")

entrance = []
attractions = [] # List to store Attraction objects
utilities = []
seasonals = []


for index, row in attraction_generator_df.iterrows():
    if row['type'] in ['Entrance']:
        new_entrance = Entrance()
        entrance.append(new_entrance)

    if row['type'] in ['Ride']:
        new_attraction = Attraction(name=row['name'], ride_duration=row['duration'], ride_capacity=row['capacity']) # change according to the finalised csv
        attractions.append(new_attraction) 
    
    if row['type'] == 'Seasonal':
        time_string = row['timeslots']
        time_list = time_string.split(", ") 
        datetime_list = [datetime.strptime(item, "%H:%M") for item in time_list]

        new_seasonal = Seasonal(name=row['name'], ride_duration=row['duration'], ride_capacity=row['capacity'], timeslot=datetime_list) # change according to the finalised csv
        seasonals.append(new_seasonal)     
    
    if row['type'] in ['Restroom', 'Dining Outlet', 'Food Cart', 'Retail']:
        new_utility = Utility(name=row['name'], service_duration=row['duration'], util_capacity=row['capacity'])
        utilities.append(new_utility)

combined_dict = {item.name: item for item in entrance + attractions + utilities + seasonals}

In [17]:
#STEP 4: IMPORTING EDGES (to facilitate Visitors travelling from one node to another)
paths_df = pd.read_csv('../data/theme_park_paths.csv')

In [18]:
#STEP 5: CREATING USS, PUT THINGS TOGETHER

# Creating ThemePark object
USS = ThemePark(spawning_dict, all_possible_itineraries)

for item in entrance:
    USS.add_entrance(item)

# Add Attraction objects to the park
for item in attractions:
    USS.add_attraction(item)

# Add Utility objects to the park
for item in utilities:
    USS.add_utility(item)

# Add Seasonal objects to the park
for item in seasonals:
    USS.add_seasonal(item)

# Open the park at 10:00
USS.current_time = datetime.strptime("09:59", "%H:%M")
USS.open_park(USS.current_time)

# Create a dictionary to store data of waiting time and crowd level across attractions and utilities across the theme park
park_data = {}

In [19]:
#STEP 6: UP AND RUNNING

while USS.current_time <= datetime.strptime("21:00", "%H:%M"): 
    # advance time on all objects:
    USS.advance_time()
    
    current_time_str = USS.current_time.strftime('%H:%M')
    if current_time_str in USS.spawning_dict.keys():        
        new_visitors = []
        result = USS.spawn_visitor(USS.current_time)
        if result != None: 
            new_visitors.extend(result)
        if new_visitors:
            for visitor in new_visitors: # spawning
                USS.existing_visitors.append(visitor)
                
    USS.existing_visitors = [visitor for visitor in USS.existing_visitors if visitor.status != "completed"]

    for item in USS.attractions + USS.seasonals + USS.utilities + USS.existing_visitors:
        item.advance_time()
    
    for visitor in USS.existing_visitors:
        if visitor.status == "done":
            visitor.find_next_location()
            visitor.find_count_down()
        elif visitor.status == "moving":
            if visitor.count_down > 0:
                visitor.count_down -= 1
            else: # arrive
                visitor.status = "none"
                visitor.current_location = visitor.next_location
        elif visitor.status == "queuing":
            if visitor.queuing > 45: # cross threshold, will move to the next itinerary
                visitor.status = "done"
                visitor.queuing = 0
            else:
                visitor.queuing += 1
        elif visitor.status == "being served":
            pass
        
        if visitor.status == "none":
            node = combined_dict[visitor.current_location]
            node.add_visitor(visitor)
    
    for node in USS.entrance + USS.attractions + USS.seasonals + USS.utilities:
        if type(node).__name__ in ["Entrance"]:
            node.process()

        if type(node).__name__ in ["Attraction", "Seasonal"]:
            node.process_ride()
            
            time_slot, crowd, wait = node.get_data() # Note: should only be called AFTER calling process_queue()
            park_data.setdefault(node.name, {}).setdefault(time_slot, {'crowd level': None, 'waiting time': None})
            park_data[node.name][time_slot]['crowd level'] = crowd
            park_data[node.name][time_slot]['waiting time'] = wait

        if type(node).__name__ in ["Seasonal"]:
            node.process_ride()
            
            time_slot, crowd, wait = node.get_data() # Note: should only be called AFTER calling process_queue()
            park_data.setdefault(node.name, {}).setdefault(time_slot, {'crowd level': None, 'waiting time': None})
            park_data[node.name][time_slot]['crowd level'] = crowd
            park_data[node.name][time_slot]['waiting time'] = wait
            
            node.release_visitor()
        
        if type(node).__name__ in ["Utility"]:
            node.process()

            time_slot, crowd, wait = node.get_data() # Note: should only be called AFTER calling process_queue()
            park_data.setdefault(node.name, {}).setdefault(time_slot, {'crowd level': None, 'waiting time': None})
            park_data[node.name][time_slot]['crowd level'] = crowd
            park_data[node.name][time_slot]['waiting time'] = wait

            


The time now at Starbucks is 1900-01-01 10:00:00.
The time now at Mel's Drive-In is 1900-01-01 10:00:00.
The time now at KT's Grill is 1900-01-01 10:00:00.
The time now at Loui's NY Pizza Parlor is 1900-01-01 10:00:00.
The time now at StarBot Cafe is 1900-01-01 10:00:00.
The time now at Oasis Spice Cafe is 1900-01-01 10:00:00.
The time now at Discovery Food Court is 1900-01-01 10:00:00.
The time now at Fossil Fuels is 1900-01-01 10:00:00.
The time now at Friar's Good Food is 1900-01-01 10:00:00.
The time now at Goldilocks is 1900-01-01 10:00:00.
The time now at Star Snacks is 1900-01-01 10:00:00.
The time now at Pops! Popcorn Delight is 1900-01-01 10:00:00.
The time now at Galactic Treats is 1900-01-01 10:00:00.
The time now at Frozen Fuel is 1900-01-01 10:00:00.
The time now at Cairo Market is 1900-01-01 10:00:00.
The time now at Pharaoh's Dessert Oasis is 1900-01-01 10:00:00.
The time now at Mariner's Market is 1900-01-01 10:00:00.
The time now at Jungle Bites is 1900-01-01 10:00:00.

NameError: name 'attraction_generator_df' is not defined

In [ ]:
# optional
#STEP 7: HOW MANY VISITORS VISITED

# Upon finishing the time period, data on number of visitors served can be retrieved
quantity_data = {}

for item in USS.attractions + USS.utilities + USS.seasonals:
    quantity_data[item.name] = item.total_served
    print(item, 'has served ', quantity_data[item.name], ' visitors.')

Accelerator has served  1055  visitors.
Battlestar Galactica: CYLON has served  286  visitors.
Battlestar Galactica: HUMAN has served  276  visitors.
Canopy Flyer has served  271  visitors.
Dino Soarin' has served  212  visitors.
Enchanted Airways has served  278  visitors.
Jurassic Park Rapids Adventure has served  1175  visitors.
Lights Camera Action Hosted by Steven Spielberg has served  555  visitors.
Magic Potion Spin has served  238  visitors.
Puss In Boots Giant Journey has served  310  visitors.
Revenge of the Mummy has served  330  visitors.
Sesame Street Spaghetti Space Chase has served  201  visitors.
Shrek 4D Adventure has served  518  visitors.
TRANSFORMERS The Ride: The Ultimate 3D Battle has served  529  visitors.
Treasure Hunters has served  237  visitors.
('Starbucks', 50.0) has served  91  visitors.
("Mel's Drive-In", 50.0) has served  89  visitors.
("KT's Grill", 50.0) has served  73  visitors.
("Loui's NY Pizza Parlor", 50.0) has served  84  visitors.
('StarBot Cafe

In [ ]:
#STEP 8: EXPORT DATA AS CSV FILES FOR FURTHER ANALYSIS

import os
# Dictionary to store DataFrames with dynamic names
dataframes = {}

# Iterate over each node (e.g., "Accelerator")
for node, time_slot in park_data.items():
    # Initialize a list to hold rows of data for each time slot
    rows = []
    
    # Iterate over each time entry in the node's data
    for timestamp, entry in time_slot.items():
        if isinstance(entry, dict):  # Ensure entry is a dictionary (which it is)
            # Prepare a row dictionary combining the timestamp and its data
            row = {'timestamp': timestamp}
            
            # Check if 'crowd level' and 'waiting time' are integers or dictionaries
            if isinstance(entry['crowd level'], dict):
                row['fast_pass_crowd_level'] = entry['crowd level']['fast_pass_queue']
                row['regular_crowd_level'] = entry['crowd level']['regular_queue']
            else:
                # Assuming 'crowd level' is just an integer (e.g., 0)
                row['fast_pass_crowd_level'] = entry['crowd level']
                row['regular_crowd_level'] = entry['crowd level']

            if isinstance(entry['waiting time'], dict):
                row['fast_pass_waiting_time'] = entry['waiting time']['fast_pass_queue']
                row['regular_waiting_time'] = entry['waiting time']['regular_queue']
            else:
                # Assuming 'waiting time' is just an integer (e.g., 0)
                row['fast_pass_waiting_time'] = entry['waiting time']
                row['regular_waiting_time'] = entry['waiting time']
            
            rows.append(row)

    # Create DataFrame for each node
    df = pd.DataFrame(rows)
    
    # Create a dynamic key for the dictionary using the node name
    df_name = f'{node}_df'
    
    # Store the DataFrame in the dictionary
    dataframes[df_name] = df

# Get current time for appending to folder name
current_time = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')

# Create output directory with timestamp included in folder name
output_dir = f'../data/simulation_output_{current_time}/'
os.makedirs(output_dir, exist_ok=True)

# Export each DataFrame to a CSV file
for key, df in dataframes.items():
    csv_file = os.path.join(output_dir, f'{key}.csv')
    df.to_csv(csv_file, index=False)

